In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import math
import torch
import torch.nn.functional as F
import llama_cpp

from path import *

llama_model_loader: loaded meta data with 36 key-value pairs and 197 tensors from ./models/Phi-3.5-mini-instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Phi 3.5 Mini Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader: - kv   4:                           general.basename str              = Phi-3.5
llama_model_loader: - kv   5:                         general.size_label str              = mini
llama_model_loader: - kv   6:                            general.license str              = mit
llama_model_loader: - kv   7:                    

In [8]:
phrase = "Italy"
k = 4
max_depth = 5
G = phrase_graph(phrase, k, max_depth)

llama_model_loader: loaded meta data with 36 key-value pairs and 197 tensors from ./models/Phi-3.5-mini-instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Phi 3.5 Mini Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader: - kv   4:                           general.basename str              = Phi-3.5
llama_model_loader: - kv   5:                         general.size_label str              = mini
llama_model_loader: - kv   6:                            general.license str              = mit
llama_model_loader: - kv   7:                    

In [5]:
import cProfile
import pstats
from pstats import SortKey

cProfile.run("G = phrase_graph(phrase, k, max_depth)", "stats")
p = pstats.Stats("stats")
p.sort_stats(SortKey.TIME)
p.print_stats()

Thu Aug 29 17:25:27 2024    stats

         35156 function calls (35145 primitive calls) in 13.641 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      341    5.403    0.016    5.403    0.016 /home/marco/miniconda3/envs/epfl/lib/python3.12/site-packages/llama_cpp/_internals.py:368(get_logits)
      341    5.257    0.015    5.258    0.015 {built-in method torch.tensor}
      341    1.165    0.003    1.165    0.003 /home/marco/miniconda3/envs/epfl/lib/python3.12/site-packages/llama_cpp/_internals.py:354(decode)
      341    0.813    0.002    0.813    0.002 {method 'tolist' of 'numpy.ndarray' objects}
      341    0.676    0.002   13.550    0.040 /home/marco/Desktop/Coding/Varcode/EPFL/LLM-path-integral/path.py:16(get_next_k_tokens)
      341    0.072    0.000    0.072    0.000 {built-in method torch.topk}
      341    0.066    0.000    6.662    0.020 /home/marco/miniconda3/envs/epfl/lib/python3.12/site-packages/llama_cpp/lla